Importation des librairies nécessaires

In [15]:
import pandas as pd
import folium
import unicodedata
import random
from IPython.display import display

Importation des données

In [16]:
filename="Data\data_lieux_culturels.xlsx" #on indique le chemin vers le fichier xlsx

data_df=pd.read_excel(filename)  #on utilise la biblothèque openpyxl pour ouvrir un fichier xlsx au format dataframe

Test pour voir si les données sont correctement récupérées

In [17]:
#for i in range (0,73612):
    #if data_df.iloc[i]["libelle_geographique"]=="Marseille" and data_df.iloc[i]["type_equipement_ou_lieu"]=="Cinéma":
        #print(data_df.iloc[i]["nom"]," , ",data_df.iloc[i]["adresse_postale"])

#pas d'erreur


Def fonctions et variables qu'on utilisera dans le programme

In [18]:
#cette fonction prend en entrée une chaîne de caracttère et la retourne sans accents.
def sans_accent(ch):
    chnorm = unicodedata.normalize('NFKD', ch)
    return u"".join([c for c in chnorm if not unicodedata.combining(c)])

#cette fonction prend en entrée une liste de lignes du classeur excel et renvoie la liste des types de lieux culturels
def liste_types_ville_def(liste_iloc):
    liste_types=[]      #on initialise une liste vide 
    for i in liste_iloc:    #on parcourt la liste d'entrée
        if data_df.iloc[i]["type_equipement_ou_lieu"] not in liste_types:   #si l'élement n'est pas dans la liste crée au dessus
            liste_types.append(data_df.iloc[i]["type_equipement_ou_lieu"])  #on l'ajoute à la liste
    return liste_types

#on extrait les iloc du classeur qui correspondent à la ville pour les mettre dans la liste 1
def func_valid_city(chosen_city_final, liste_iloc_first_step):
    for i in range (0,len(data_df)):    #on parcourt toutes les colonnes
        ville=str(data_df.iloc[i]["libelle_geographique"])  #on extrait la ville de la colonne i
        ville=sans_accent(ville).upper()    #on la rend insensible à la casse
        ville_no_hyphen=ville.replace("-"," ")  #on crée une 2e variable qui prend en valeur la ville à l'index i sans les tirets (s'il y en a)
        
        if chosen_city_final==ville or chosen_city_final==ville_no_hyphen:  #si il y a correspondance entre la ville d'entrée et la ville à l'index i
            liste_iloc_first_step.append(i)     #on ajout l'index à la liste

#on extrait les iloc de la liste 1 les iloc qui correspondent au type pour les mettre dans la liste 2
def func_valid_type(liste_iloc_first_step, chosen_type_final, liste_iloc_second_step):
    for i in liste_iloc_first_step:     #on parcourt la liste 1
        type_lieu=str(data_df.iloc[i]["type_equipement_ou_lieu"])   #on extrait le type correspondant à l'index i de la liste 1
        type_lieu=sans_accent(type_lieu).upper() #on le rend insensible à la casse

        if chosen_type_final==type_lieu:     #si le type à l'index i correspond àau type d'entrée de notre fonction
            liste_iloc_second_step.append(i)    #on ajoute l'index i à la liste 2

#cette fonction prend en entrée une liste de lignes du classeur excel et retourne une map avec des marqueurs sur les coordonnées correspondantes
def affichage_map(list_iloc):
    geoloc=[]   #on initialise une liste vide
    for i in list_iloc:    #on parcourt la liste de d'entrée
        coordonnee_geo=data_df.iloc[i]["coordonnees_geo"].split(",")    #on récupère les coordonées gps correspondant à un lieux culturels qu'on va mettre dans une liste temporaire
        lat,lon=coordonnee_geo[0],coordonnee_geo[1]     #on affecte lat et lon à la latitude et longitude
        coordinates=(lat,lon)           #on crée un tuple avec lat et lon
        geoloc.append(coordinates)      #on ajoute le tuple à la liste crée au départ de la fonction
    centre=geoloc[0]        #on définit le centre de la map comme la premier tuple de coordonnées (pratique quand on a qu'une coordonnée)
    print_map=folium.Map(location=centre,zoom_start=15)     #on définit notre variable print_map comme une map folium, avec un centre et un niveau de zoom (modifiable avec la souris après affichage)
    for i in range(0,len(geoloc)):      #on parcourt tous les tuples de coordonéées
        folium.Marker(geoloc[i],popup=data_df.iloc[list_iloc[i]]["nom"]).add_to(print_map)      #on ajoute un marqueur sur la map pour chaque tuple
    display(print_map)      #on force l'affichage de la map



Code programme

In [ ]:
entry=""

while entry!="q":   #boucle pour pouvoir gérer un arrêt du programme
    print("Bonjour, je suis une IA destinée à vous informer sur les lieux culturels proche d'un emplacement.\nVous pouvez entrer à tout moment 'q' pour quitter.")

    #première étape : filtrer les données avec la ville que renseigne l'utilisateur
    valid_city=False

    while valid_city==False:    
        chosen_city=input(print("Pour commencer, merci d'indiquer une ville."))

        #gestion du quit
        if chosen_city=="q":
            print("Au revoir !")
            break
        
        #on rend la ville reneignée par l'utilisateur insensible à la casse
        chosen_city_up=chosen_city.upper()
        chosen_city_final=sans_accent(chosen_city_up)

        liste_iloc_first_step=[]    #on initialise une liste vide 

        func_valid_city(chosen_city_final, liste_iloc_first_step)

        if len(liste_iloc_first_step)==0:   #si la liste est vide après exécution de la fonction
            print("Désolé, je n'ai trouvé aucune ville correspondant à",chosen_city+".")    #ça signifie que la base de données ne possède pas de ville correspondant au nom
            continue    #donc on fait redemarrer la boucle
        else:
            print("Il y a",len(liste_iloc_first_step),"lieux culturels à",chosen_city+".\n")    #sinon on affiche le nombre de lieux culturels dans la ville choisis
            valid_city=True     #on bascule sur True pour arréter la boucle

        print("Voici les différents types de lieux culturels à",chosen_city,":")
        for i in liste_types_ville_def(liste_iloc_first_step):
            print("-",i)

    if chosen_city=="q":    #gestion du quit
        break

    #deuxième étape : filtrer les données avec le type de lieux culturels que renseigne l'utilisateur
    valid_type=False
    while valid_type==False:
        chosen_type=input(print("\nMerci d'indiquer le type de lieu culturel qui vous intéresse."))
        chosen_type_up=chosen_type.upper()
        chosen_type_final=sans_accent(chosen_type_up)

        #gestion du quit
        if chosen_type=="q":
            print("Au revoir !")
            break

        liste_iloc_second_step=[]

        func_valid_type(liste_iloc_first_step, chosen_type_final, liste_iloc_second_step)
        third_step_choice_value=""
        if len(liste_iloc_second_step)>1: #si il y a plusieurs lieux qui valident ville + type
            print("Il y a",len(liste_iloc_second_step),chosen_type,"à",chosen_city+".") #on affiche combien il y en a
            valid_type=True #on met valid_type sur True pour arréter la boucle
            print("Voulez vous me donner un mot clé pour affiner la recherche ? Si non j'en séléctionnerai un au hasard parmis les",len(liste_iloc_second_step),".","Sinon je peux tous les afficher.")
            while (third_step_choice_value!="oui" and third_step_choice_value!="non" and third_step_choice_value!="tous"): #tant que l'utilisateur n'a pas répondu correctement
                third_step_choice_value=input(print("(oui/non/tous)")).lower() #insensible à la casse

                if third_step_choice_value=="q":#gestion du quit
                    break

        if third_step_choice_value=="q":
            print ("Au revoir !")
            break
        elif len(liste_iloc_second_step)==0: #il n'y a aucun lieu qui valide ville + type
            print("\nDésolé je n'ai trouvé aucun lieu culturel du type",chosen_type,"à",chosen_city)
            continue    #on recommence la boucle
        elif len(liste_iloc_second_step)==1:    #si il n'y en a qu'un seul
            valid_type=True    #on met valid_type sur True pour arréter la boucle
            print("Le seul",chosen_type,"à",chosen_city,"est",data_df.iloc[liste_iloc_second_step[0]]["nom"],"a l'adresse :",'"',data_df.iloc[liste_iloc_second_step[0]]["adresse_postale"],'"')
            third_step_choice_value="only_one"  #on affecte la variable (elle servira après)
    
    if chosen_type=="q" or third_step_choice_value=="q":    #gestion du quit
        break
    
    #troisième étape : filtrer les données avec un éventuel mot-clé renseigné par l'utilisateur
    liste_iloc_third_step=[]
    keyword=""

    if third_step_choice_value=="oui":  #l'utilisateur souhaite indiquer un mot-clé
        keep_third_step=True
        while keep_third_step:
            keyword=input(print("Saisissez un mot-clé pour affiner votre recherche"))  

            if keyword=="q":    #gestion du quit
                break

            keyword_final=sans_accent(keyword).upper()

            for i in liste_iloc_second_step:    #ici on extrait de les lieux qui correspondent au mot clé
                if keyword_final in sans_accent(str(data_df.iloc[i]["nom"])).upper() or keyword_final in sans_accent(str(data_df.iloc[i]["domaine"])).upper() or keyword_final in sans_accent(str(data_df.iloc[i]["sous_domaines"])).upper():   #on cherche le mot clé dans différentes colonnes
                    liste_iloc_third_step.append(i) #si il y a correspondance on ajoute l'index

            if len(liste_iloc_third_step)>0:    #si il y a eu des correspondance
                keep_third_step=False   #on arrête la boucle
                keyword_recognized=True    #on indique le le mot-clé a été reconnu
            #dans le cas ou il n'a pas été reconnu :
            else:
                keep_third_step_test=""
                while keep_third_step_test!="oui" and keep_third_step_test!="non":  #on demande à l'utilisateur s'il veut réessayer
                    keep_third_step_test=input(print("Désolé, aucun lieu ne correspond à votre mot clé.\n Voulez vous réessayer un autre mot ?(oui/non)"))
                if keep_third_step_test=="oui":
                    keep_third_step=True    #si oui on maintient la boucle
                elif keep_third_step_test=="non":
                    keep_third_step=False   #si non on arrête la boucle
                    keyword_recognized=False    #et on indique que le mot-clé n'a pas été reconnu

        if keyword!="q":    #si l'utilisateur n'a pas demandé a arrêter le programme
            if keyword_recognized:  #si le mot-clé a été reconnu
                for i in liste_iloc_third_step:
                    print("Vous trouverez le",chosen_type,data_df.iloc[i]["nom"],"à l'adresse",data_df.iloc[i]["adresse_postale"])  #on indique tous les lieux qui valident ville + type + mot clé

            elif not keyword_recognized:
                print("Voici la liste des",chosen_type,"à",chosen_city,":")     #sinon on indique les lieux qui valident ville + type
                for i in liste_iloc_second_step:
                    print("-",data_df.iloc[i]["nom"],":",data_df.iloc[i]["adresse_postale"])

    if keyword=="q":    #gestion du quit
        print("Au revoir !")
        break

    elif third_step_choice_value=="non":    #si l'utilisateur ne veut pas renseigner de mot clé
        rand_iloc=random.randint(0,len(liste_iloc_second_step)) #on selectionne un index au hasard
        print("Voici un",chosen_type,"à",chosen_city,":\n-",data_df.iloc[rand_iloc]["nom"],":",data_df.iloc[rand_iloc]["adresse_postale"])
        liste_iloc_third_step.append(rand_iloc) #on ajout l'index aléatoire à la liste d'index de la troisième étape

    elif third_step_choice_value=="tous" and len(liste_iloc_second_step)>1: #si l'utilisateur veut afficher tous les lieux qui valident ville + type
        print("Voici la liste des",chosen_type,"à",chosen_city,":")
        for i in liste_iloc_second_step:
            print("-",data_df.iloc[i]["nom"],":",data_df.iloc[i]["adresse_postale"])
        liste_iloc_third_step=liste_iloc_second_step    #on met à jour la liste d'index de la troisième étape
        
    elif third_step_choice_value=="only_one":   #si il n'y a qu'un lieu qui valident ville + type
        liste_iloc_third_step=liste_iloc_second_step    #on met à jour la liste d'index de la troisième étape

    #quatrième étape : on demande à l'utilisateur s'il veut afficher une carte avec les emplacements des lieux
    bool_affichage=""
    while bool_affichage !="oui" and bool_affichage!="non":
        bool_affichage=input(print("Voulez vous afficher une carte avec les emplacements ?\n(oui/non)")).lower()

        if bool_affichage=="q": #gestion du quit
            break

    if bool_affichage=="oui":   #si oui
        affichage_map(liste_iloc_third_step)    #on affiche
        print("Au revoir")
        break
    elif bool_affichage=="non" or bool_affichage=="q":  #si non, ou que l'utilisateur a demandé à quitter le programme
        print("Au revoir !")    #on n'affiche pas la carte
        break
